In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json, os

In [2]:
def make_df(path_to_file, inp_size, out_size,num_data=500, examples=2):
  #inp_size, out_size, examples = 15, 15, 4      # Modify
  cols = (inp_size**2 + out_size**2) * examples
  df = pd.DataFrame(columns=list(range(cols))+['operation','output'])
  op_list = [f'Dilation SE{i}' for i in range(1,9)] + [f'Erosion SE{i}' for i in range(1,9)]
  #path_to_file = "/content/IPARC_ChallengeV2/Dataset/CatA_Simple"   # Modify

  for id in range(num_data):
    with open(f'{path_to_file}/Task{id:03}.json') as f:
      data = json.load(f)
    with open(f'{path_to_file}/Task{id:03}_soln.txt') as file:
      lines = [line.rstrip() for line in file]
    d = []
    for i in range(len(data)):
      d += [element for innerList in data[i]['input'] for element in innerList] + [element for innerList in data[i]['output'] for element in innerList]
    for op in op_list:
      df.loc[len(df)] = (d + [op, op in lines])
  
  return df

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

def make_X_y(df):
    
    y = df.pop('output')
    X = df
    X = pd.get_dummies(df, columns = ['operation'])
    X.columns = X.columns.astype(str)
    
    print(f"Percentage of positive labels in out dataset: {100*sum(y == 1)/len(y)}%")
    return X, y

def training(X, y, test_size=0.3):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    perform_dict = dict()
    model_list = [('logistic_regression', LogisticRegression(random_state=0)),
                  ('decision_tree', DecisionTreeClassifier(random_state=0)),
                  ('support_vector_classifier', SVC(gamma='auto', random_state=0)),
                  ('random_forest', RandomForestClassifier(max_depth=2, random_state=0)),
                  ('gradient_boosting', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
                 ]
    
    for name, model in model_list:
        print(name)
        clf = model.fit(X_train, y_train)
        train_acc, test_acc = clf.score(X_train, y_train), clf.score(X_test, y_test)
        y_test_predictions = clf.predict(X_test)
        precision = precision_score(y_test, y_test_predictions)
        recall = recall_score(y_test, y_test_predictions)
        f1score = f1_score(y_test, y_test_predictions)
        
        perform_dict[f'{name}_train_accuracy'], perform_dict[f'{name}_test_accuracy'] = train_acc, test_acc
        perform_dict[f'{name}_precision'] = precision
        perform_dict[f'{name}_recall'] = recall
        perform_dict[f'{name}_f1'] = f1score
        
        print(f'    train accuracy = {clf.score(X_train, y_train)}')
        print(f'    test accuracy = {clf.score(X_test, y_test)}')
        print(f"    % '1's predicted: {100*sum(y_test_predictions == 1)/len(y_test_predictions)}%")
        print(f"    Precision = {precision}")
        print(f"    Recall = {recall}")
        print(f"    F1 Score = {f1score}\n\n")
    
    return perform_dict
    

In [6]:
for seq in range(2, 11):
    df = make_df(f'../Dataset/Sequence_{seq}/CatA_Simple', 15, 15, num_data=500)
    print(f"For Datasets generated with {seq} number of operations:")
    
    X, y = make_X_y(df)
    training(X, y, test_size=0.3)

For Datasets generated with 2 number of operations:
Percentage of positive labels in out dataset: 23.075%
logistic_regression


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    train accuracy = 0.7714285714285715
    test accuracy = 0.76125
    % '1's predicted: 0.3333333333333333%
    Precision = 0.0
    Recall = 0.0
    F1 Score = 0.0


decision_tree
    train accuracy = 1.0
    test accuracy = 0.6541666666666667
    % '1's predicted: 23.291666666666668%
    Precision = 0.2629695885509839
    Recall = 0.26017699115044246
    F1 Score = 0.26156583629893243


support_vector_classifier


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    train accuracy = 0.77125
    test accuracy = 0.7645833333333333
    % '1's predicted: 0.0%
    Precision = 0.0
    Recall = 0.0
    F1 Score = 0.0


random_forest


...

    train accuracy = 0.6978571428571428
    test accuracy = 0.6508333333333334
    % '1's predicted: 88.04166666666667%
    Precision = 0.6862281116895409
    Recall = 0.8923076923076924
    F1 Score = 0.7758159443552701


decision_tree
    train accuracy = 1.0
    test accuracy = 0.7008333333333333
    % '1's predicted: 70.375%
    Precision = 0.76850207223209
    Recall = 0.7987692307692308
    F1 Score = 0.7833433916716959


support_vector_classifier
    train accuracy = 0.6730357142857143
    test accuracy = 0.6770833333333334
    % '1's predicted: 100.0%
    Precision = 0.6770833333333334
    Recall = 1.0
    F1 Score = 0.8074534161490683


random_forest
    train accuracy = 0.6730357142857143
    test accuracy = 0.6770833333333334
    % '1's predicted: 100.0%
    Precision = 0.6770833333333334
    Recall = 1.0
    F1 Score = 0.8074534161490683


gradient_boosting
    train accuracy = 0.6778571428571428
    test accuracy = 0.66375
    % '1's predicted: 92.58333333333333%
    Preci

C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    train accuracy = 0.7558928571428571
    test accuracy = 0.6929166666666666
    % '1's predicted: 79.16666666666667%
    Precision = 0.7547368421052632
    Recall = 0.8410557184750733
    F1 Score = 0.7955617198335645


decision_tree
    train accuracy = 1.0
    test accuracy = 0.74625
    % '1's predicted: 71.16666666666667%
    Precision = 0.8208430913348946
    Recall = 0.8222873900293255
    F1 Score = 0.8215646059185467


support_vector_classifier
    train accuracy = 0.7105357142857143
    test accuracy = 0.7104166666666667
    % '1's predicted: 100.0%
    Precision = 0.7104166666666667
    Recall = 1.0
    F1 Score = 0.830694275274056


random_forest
    train accuracy = 0.7105357142857143
    test accuracy = 0.7104166666666667
    % '1's predicted: 100.0%
    Precision = 0.7104166666666667
    Recall = 1.0
    F1 Score = 0.830694275274056


gradient_boosting
    train accuracy = 0.7371428571428571
    test accuracy = 0.6941666666666667
    % '1's predicted: 85.29166666666667

C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    train accuracy = 0.7994642857142857
    test accuracy = 0.72875
    % '1's predicted: 71.75%
    Precision = 0.8042973286875726
    Recall = 0.8151854031783402
    F1 Score = 0.8097047646886875


decision_tree
    train accuracy = 1.0
    test accuracy = 0.7575
    % '1's predicted: 70.45833333333333%
    Precision = 0.8302779420461266
    Recall = 0.8263684520306063
    F1 Score = 0.8283185840707965


support_vector_classifier
    train accuracy = 0.7148214285714286
    test accuracy = 0.7079166666666666
    % '1's predicted: 100.0%
    Precision = 0.7079166666666666
    Recall = 1.0
    F1 Score = 0.8289826787021224


random_forest
    train accuracy = 0.7148214285714286
    test accuracy = 0.7079166666666666
    % '1's predicted: 100.0%
    Precision = 0.7079166666666666
    Recall = 1.0
    F1 Score = 0.8289826787021224


gradient_boosting
    train accuracy = 0.7769642857142857
    test accuracy = 0.7341666666666666
    % '1's predicted: 75.125%
    Precision = 0.7942318358291

In [ ]:
data_size = dict()
for size in range(9,24,2):
    print(f"For Datasets generated with Input and Output grid of size {size}:\n")
    df = make_df(f'../Dataset/size_{size:02}', size, size, num_data=500)
    X, y = make_X_y(df)
    data_size[size] = training(X, y, test_size=0.3)

For Datasets generated with Input and Output grid of size 9:

Percentage of positive labels in out dataset: 43.55%
logistic_regression


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    train accuracy = 0.6066071428571429
    test accuracy = 0.5416666666666666
    % '1's predicted: 28.916666666666668%
    Precision = 0.4668587896253602
    Recall = 0.30740037950664134
    F1 Score = 0.3707093821510297


decision_tree
    train accuracy = 1.0
    test accuracy = 0.6808333333333333
    % '1's predicted: 43.833333333333336%
    Precision = 0.6368821292775665
    Recall = 0.635673624288425
    F1 Score = 0.6362773029439696


support_vector_classifier


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    train accuracy = 0.5667857142857143
    test accuracy = 0.5608333333333333
    % '1's predicted: 0.0%
    Precision = 0.0
    Recall = 0.0
    F1 Score = 0.0


random_forest


...

    train accuracy = 0.6035714285714285
    test accuracy = 0.45916666666666667
    % '1's predicted: 42.125%
    Precision = 0.410484668644906
    Recall = 0.37153088630259623
    F1 Score = 0.39003759398496235


decision_tree
    train accuracy = 1.0
    test accuracy = 0.5279166666666667
    % '1's predicted: 47.833333333333336%
    Precision = 0.4930313588850174
    Recall = 0.5067144136078783
    F1 Score = 0.4997792494481236


support_vector_classifier
    train accuracy = 0.5323214285714286
    test accuracy = 0.5270833333333333
    % '1's predicted: 2.5833333333333335%
    Precision = 0.3548387096774194
    Recall = 0.019695613249776187
    F1 Score = 0.037319762510602206


random_forest


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


    train accuracy = 0.5226785714285714
    test accuracy = 0.5345833333333333
    % '1's predicted: 0.0%
    Precision = 0.0
    Recall = 0.0
    F1 Score = 0.0


gradient_boosting
    train accuracy = 0.58125
    test accuracy = 0.4979166666666667
    % '1's predicted: 40.75%
    Precision = 0.4550102249488753
    Recall = 0.39838854073410923
    F1 Score = 0.4248210023866349


For Datasets generated with Input and Output grid of size 19:

Percentage of positive labels in out dataset: 47.975%
logistic_regression


C:\Users\TEJAS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


    train accuracy = 0.60375
    test accuracy = 0.4583333333333333
    % '1's predicted: 43.875%
    Precision = 0.42924976258309594
    Recall = 0.39270199826238056
    F1 Score = 0.41016333938294014


decision_tree
    train accuracy = 1.0
    test accuracy = 0.5325
    % '1's predicted: 46.791666666666664%
    Precision = 0.5129118432769367
    Recall = 0.5004344048653345
    F1 Score = 0.5065963060686015


support_vector_classifier
    train accuracy = 0.5401785714285714
    test accuracy = 0.5025
    % '1's predicted: 7.208333333333333%
    Precision = 0.37572254335260113
    Recall = 0.05647263249348393
    F1 Score = 0.09818731117824774


random_forest
    train accuracy = 0.5203571428571429
    test accuracy = 0.52
    % '1's predicted: 0.041666666666666664%
    Precision = 0.0
    Recall = 0.0
    F1 Score = 0.0


gradient_boosting
    train accuracy = 0.5816071428571429
    test accuracy = 0.47541666666666665
    % '1's predicted: 44.333333333333336%
    Precision = 0.449248

In [ ]:
import matplotlib.pyplot as plt
model_list = ['logistic_regression', 'decision_tree', 'support_vector_classifier', 'random_forest', 'gradient_boosting']
                 
x = list(range(2,11))

for model in model_list:
    y = [data_dict[size][f'{name}_test_accuracy'] for size in range(2, 11)]
    plt.plot(x,y, label=model)
plt.xlabel('No. of sequence')
plt.ylabel('Test accuracy')
plt.title('Test accuracies')
plt.show()

In [ ]:
df = pd.DataFrame()
x = list(range(2,11))
df['size'] = x
for model in model_list:
    y = [data_dict[size][f'{name}_test_accuracy'] for size in range(2, 11)]
    df[model]  = y
df.to_csv("test_acc.csv")